1. Download hagmann connectome from TheVirtualBrain github
2. Unzip the package
3. Build NetworkX graph

In [22]:
import requests

url = 'https://github.com/the-virtual-brain/tvb-data/raw/master/tvb_data/connectivity/connectivity_998.zip'

r = requests.get(url)  
with open('./connectivity_998.zip', 'wb') as f:
    f.write(r.content)

In [2]:
from shutil import unpack_archive
unpack_archive('./connectivity_998.zip', 'connectivity_998/')

In [31]:
import numpy as np
# read nodes region names and centers
centres = np.genfromtxt('connectivity_998/centres.txt',
                        dtype='|U256,i,i,i',
                        encoding='utf-8',
                        names=['name','x','y','z'])

# convert nodes region names to dict
regions_dict = dict()
positions_dict = dict()
for idx, r in enumerate(centres):
    regions_dict[idx] = r['name']
    positions_dict[idx] = (r['x'], r['y'],r['z'])
    
    
weights = np.genfromtxt('connectivity_998/weights.txt')


In [54]:
import networkx as nx

# create a graph from weighted adjacency matrix
weights_graph = nx.from_numpy_matrix(weights)

# create a graph from nodes' positions
graph = nx.Graph()
graph.add_nodes_from(positions_dict.keys())

for n, p in positions_dict.items():
    graph.node[n]['x'] = p[0]
    graph.node[n]['y'] = p[1]
    graph.node[n]['z'] = p[2]
    graph.node[n]['region_id'] = 6
    graph.node[n]['label'] = 'lof1Res1015_RH'
    graph.node[n]['fs_structname'] = 'lateralorbitofrontal1'
# copy weighted edges
graph.add_weighted_edges_from(graphw.edges(data=True))

# graph = nx.relabel_nodes(graph, names_dict)
# nx.set_node_attributes
# print(graph.nodes(data=True))
#nx.draw_networkx(graph, pos=positions_dict)
nx.write_gpickle(graph,'/Volumes/_Shared/15.Applications/brainX3/Datasets/specs-bids/epilepsy/sub-conn/anat/Broadmann_998_2.gpickle')

[(0, {'x': 29, 'y': 26, 'z': -1, 'region_id': 6, 'label': 'lof1Res1015_RH', 'fs_structname': 'lateralorbitofrontal1'}), (1, {'x': 36, 'y': 27, 'z': -14, 'region_id': 6, 'label': 'lof1Res1015_RH', 'fs_structname': 'lateralorbitofrontal1'}), (2, {'x': 28, 'y': 24, 'z': -9, 'region_id': 6, 'label': 'lof1Res1015_RH', 'fs_structname': 'lateralorbitofrontal1'}), (3, {'x': 28, 'y': 19, 'z': -16, 'region_id': 6, 'label': 'lof1Res1015_RH', 'fs_structname': 'lateralorbitofrontal1'}), (4, {'x': 19, 'y': 14, 'z': -17, 'region_id': 6, 'label': 'lof1Res1015_RH', 'fs_structname': 'lateralorbitofrontal1'}), (5, {'x': 15, 'y': 19, 'z': -17, 'region_id': 6, 'label': 'lof1Res1015_RH', 'fs_structname': 'lateralorbitofrontal1'}), (6, {'x': 17, 'y': 25, 'z': -18, 'region_id': 6, 'label': 'lof1Res1015_RH', 'fs_structname': 'lateralorbitofrontal1'}), (7, {'x': 15, 'y': 26, 'z': -16, 'region_id': 6, 'label': 'lof1Res1015_RH', 'fs_structname': 'lateralorbitofrontal1'}), (8, {'x': 21, 'y': 29, 'z': -10, 'region_

In [ ]:
!cat connectivity_998/weights.txt

In [ ]:
!ls connectivity_998

In [34]:
q = nx.read_gpickle('/Volumes/_Shared/15.Applications/brainX3/Datasets/specs-bids/epilepsy/sub-conn/anat/Broadmann_998.gpickle')

In [35]:
print(q.nodes(data=True))

[('n1', {'x': -17.223499298095703, 'y': 24.24340057373047, 'z': -17.195899963378906, 'region_id': 0, 'label': 'lof1Res1015_RH', 'fs_structname': 'lateralorbitofrontal1'}), ('n2', {'x': -27.899799346923828, 'y': 23.321699142456055, 'z': -18.472700119018555, 'region_id': 0, 'label': 'lof1Res1015_RH', 'fs_structname': 'lateralorbitofrontal2'}), ('n3', {'x': -28.917400360107422, 'y': 18.039499282836914, 'z': -13.564900398254395, 'region_id': 0, 'label': 'lof1Res1015_RH', 'fs_structname': 'lateralorbitofrontal3'}), ('n4', {'x': -30.26180076599121, 'y': 20.127099990844727, 'z': -4.227680206298828, 'region_id': 0, 'label': 'lof1Res1015_RH', 'fs_structname': 'lateralorbitofrontal4'}), ('n5', {'x': -33.38330078125, 'y': 27.98349952697754, 'z': -4.7047200202941895, 'region_id': 0, 'label': 'lof1Res1015_RH', 'fs_structname': 'lateralorbitofrontal5'}), ('n6', {'x': -41.2958984375, 'y': 22.047300338745117, 'z': -11.644599914550781, 'region_id': 0, 'label': 'lof1Res1015_RH', 'fs_structname': 'latera